# Praca Domowa 3
Bartosz Siński 

In [ ]:
import pandas as pd
import numpy as np

### Załadowanie modelu i podział zbioru na treningowy i testowy.

In [ ]:
from sklearn.model_selection import train_test_split
df_aus = pd.read_csv("./src/australia.csv")
X= df_aus.drop(["RainTomorrow"], axis=1)
y = df_aus["RainTomorrow"]
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y,random_state = 42)

### Regresja logistyczna

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
lreg = LogisticRegression(random_state=1613,penalty='l1',solver='saga',max_iter=500).fit(X_train, y_train)
aus_pred1 = lreg.predict(X_test)
accuracy_score(aus_pred1,y_test)

Dobraliśmy parametry *penalty, max_iter* oraz *solver*. *Penalty* ustawia rodzaj kary, którą nakładamy na model za overfitting. Parametr *Solver* odpowiada za wybór algorytmu opdowiedzialnego za optymalizacje, a *max_iter* za ograniczenie liczby iteracji naszego solvera. Wybraliśmy *saga* ponieważ pozwala on na wybranie parametru regularyzacji L1 i działa szybko na dużych zbiorach danych. Zmiana kary na L1 i zwiększenie liczby iteracji nieznacznie poprawiło accuracy naszego modelu.

### SVM 

In [ ]:
from sklearn.svm import SVC
svm = SVC(random_state=1613,kernel = 'linear',C=10)
svm.fit(X_train, y_train)
aus_pred2 = svm.predict(X_test)
accuracy_score(aus_pred2,y_test)

W powyższym modelu SVC ustawiliśmy parametr *kernel*, który ustala typ jądra używanego w algorytmie na liniowy. Zmieniliśmy także parametr regularyzacji *C* z domyślnego 1 na 10. W obu przypadkach zmiana parametru podniosła nasze accuracy.

### Random Forrest 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=1613,n_estimators = 1000,max_features='log2')
rfc.fit(X_train,y_train)
aus_pred3 = rfc.predict(X_test)
accuracy_score(aus_pred3,y_test)

W modelu Random Forrest zmieniliśmy domyślne wartości parametrów *n_estimators* i *max_features*. *n_estimators* ustala liczbę drzew w naszym lesie na podstawie których będziemy przewidywać wartość targetu. Parametr *max_features* odpowiada za liczbę zmiennych przy podziałach liści. Wartość *n_estimators* podnieśliśmy z 100 do 1000 co zwiększyło czas przygotowania modelu jednak podniosło też jego accuracy. *max_features* zmieniliśmy z domyślnej *sqrt* na *log2* co nie wpłyneło na predykcje naszego modelu.

### Porównanie wyników

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
results = {
    "algorithm" : ['Logistic Regression','SVM','Random Forrest'],
    "accuracy" : [accuracy_score(y_test,aus_pred1),accuracy_score(y_test,aus_pred2),accuracy_score(y_test,aus_pred3)],
    "precision" : [precision_score(y_test,aus_pred1),precision_score(y_test,aus_pred2),precision_score(y_test,aus_pred3)],
    "recall" :[recall_score(y_test,aus_pred1),recall_score(y_test,aus_pred2),recall_score(y_test,aus_pred3)],
    'ROC AUC' : [roc_auc_score(y_test,aus_pred1),roc_auc_score(y_test,aus_pred2),roc_auc_score(y_test,aus_pred3)],
    'F1' : [f1_score(y_test,aus_pred1),f1_score(y_test,aus_pred2),f1_score(y_test,aus_pred3)]

}
pd.DataFrame(results)

Najlepszym klasyfikatorem okazał się Random Forest. We wszystkich metrykach osiągnął lepszy wynik niż pozstałe modele. Ciekawe wydaje się być, że SVM i Regresja Logistyczna osiągneły bardzo podobne wyniki we wszystkich metrykach.Dodatkowo zaskakujący jest tak wysoki wynik Regresji logistycznej w porównaniu do bardziej zaawansowanych modeli SVM i Random Forrest. Spojrzymy także na *table of confusion* dla tych modeli.

### Regresja Logistyczna

In [ ]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, aus_pred1).ravel()
pd.DataFrame({"Actual positives": [tp, fp], "Actual negatives": [fn, tn]}, index = ["Positive predictions", "Negative predictions"])

### SVM 

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, aus_pred2).ravel()
pd.DataFrame({"Actual positives": [tp, fp], "Actual negatives": [fn, tn]}, index = ["Positive predictions", "Negative predictions"])

### Random Forest

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, aus_pred3).ravel()
pd.DataFrame({"Actual positives": [tp, fp], "Actual negatives": [fn, tn]}, index = ["Positive predictions", "Negative predictions"])

Powyżej widzimy, że Random Forrest najbardziej różni się od pozostałych modeli pod względem zdecydowanie mniejszej liczby predykcji *False Negative*. Widzimy także, że modele o wiele lepiej radzą sobie z klasyfikacją obserwacji, gdzie model przewiduje brak deszczu. Może się to wiązać z dużo większą ilością obserwacji gdzie deszczu nie ma.   